In [1]:
!mkdir training_folder && cd training_folder
%cd training_folder
!git clone https://github.com/louisgeist/Generative_Models_with_Sinkhorn.git
%cd Generative_Models_with_Sinkhorn/

/content/training_folder
Cloning into 'Generative_Models_with_Sinkhorn'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 45 (delta 20), reused 30 (delta 11), pack-reused 0
Receiving objects: 100% (45/45), 12.17 KiB | 12.17 MiB/s, done.
Resolving deltas: 100% (20/20), done.
/content/training_folder/Generative_Models_with_Sinkhorn


In [2]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from gen_model_with_sinkhorn import Model

import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [3]:
torch.cuda.is_available()

True

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("My device is", device)

#PARAMETERS
batch_size = 64
epochs = 20
generator_dim = [[2,32], [32,128], [128, 784]] #last one should be [_,784]
learned_cost_dim = [[784, 128], [128, 128]] #first one should be [784, _]
lr = 0.01
learnable_cost = False
epsilon = 0.1

model = Model(generator_dim, learned_cost_dim, batch_size, lr, epsilon, learnable_cost).to(device)

#to use normalized version of MNIST
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Download and load the MNIST dataset
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)


# Flatten 28x28 image into a vector= of 784 fetures
flatten = transforms.Lambda(lambda x: x.view(-1))

# Apply the flatten transformation to the dataset
train_dataset.transform = transforms.Compose([transform, flatten])
test_dataset.transform = transforms.Compose([transform, flatten])

# Create dataloaders
train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True,pin_memory=True)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False,pin_memory=True)

My device is cuda:0


100%|██████████| 9912422/9912422 [00:00<00:00, 163329593.06it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 121135693.82it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 31342696.70it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 20662178.71it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [5]:
#training
for epoch in range(1,epochs+1):
    model.train(True)
    loss = model.train_1epoch(train_dataloader)
    print(f"epoch{epoch}: loss = {loss}\n")

torch.save(model, "basic_model.pt")


RuntimeError: ignored

In [ ]:
import visualization.py